In [12]:
!pip install pyodbc # ODBC的DRIVER(驅動程式)

Active code page: 936


### ----開始執行----

In [1]:
# coding=utf-8
from platform import python_version
import os, time, socket, glob, pickle, joblib
import pandas as pd
import pyodbc, pymssql
import warnings
warnings.filterwarnings(action="ignore")
print("【現在時間】{}".format(time.strftime("%Y/%m/%d %H:%M:%S")))
print("【所在目錄】{}".format(os.getcwd()))
#print("【主機名稱】{} ({})".format(socket.gethostname(),socket.gethostbyname(socket.
print("【Python】{}".format(python_version()))
print("【pymssql】{}".format(pymssql.__version__))
print("【pyodbc】{}".format(pyodbc.version))
%autosave 120

【現在時間】2022/12/18 03:52:47
【所在目錄】C:\Users\User\Desktop\資料工程
【Python】3.9.13
【pymssql】2.2.7
【pyodbc】4.0.35


Autosaving every 120 seconds


In [3]:
import pandas as pd
import requests
import bs4
from bs4 import BeautifulSoup

In [56]:
stock_id = "1218"  #1605 1218 6446
url = 'https://goodinfo.tw/tw/BasicInfo.asp?STOCK_ID='+stock_id
payloads = {"STOCK_ID":stock_id}
headers = {"user-agent":"Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) \
Chrome/107.0.0.0 Safari/537.36"}
resp = requests.request(method="POST",url=url,data=payloads,headers=headers)
resp.encoding = "utf-8"

info_table = pd.read_html(resp.text)

In [57]:
df = info_table[14].dropna(axis='columns')
df

,0,1,2,3
0,公 司 基 本 資 料,公 司 基 本 資 料,公 司 基 本 資 料,公 司 基 本 資 料
1,股票代號,1218,股票名稱,泰山
2,產業別,食品工業,上市/上櫃,上市
3,公司名稱,泰山企業股份有限公司,泰山企業股份有限公司,泰山企業股份有限公司
4,英文簡稱,TAISUN,TAISUN,TAISUN
5,成立日期,1950/10/26 (72年),1950/10/26 (72年),1950/10/26 (72年)
6,掛牌日期,1989/11/11 (33年),1989/11/11 (33年),1989/11/11 (33年)
7,上市日期,1989/11/11,上櫃日期,-
8,興櫃日期,-,公開發行日期,-
9,資本額,50 億元,每股面值,新台幣 10 元


In [58]:
products_list = df.values.tolist()
print(products_list[1])
print(products_list[2])
print(products_list[11])
print(products_list[13])

lists_num=len(products_list)
print(len(products_list))

['股票代號', '1218', '股票名稱', '泰山']
['產業別', '食品工業', '上市/上櫃', '上市']
['發行股數', '499,999,038\xa0股 (含私募0股)', '特別股', '0\xa0股']
['董事長', '詹景超', '詹景超', '詹景超']
33


### SQL連線

In [12]:
connsqlodbc= pyodbc.connect("DRIVER={ODBC Driver 17 for SQL Server};SERVER=124.218.194.84;"
                      "DATABASE=testDB;UID=testuser;PWD=323")


In [50]:
connsqlodbc.autocommit = True
cursor = connsqlodbc.cursor()
# GS+代號+發周別
# MYSQL指令
time_S=time.strftime("%Y/%m/%d")
print(time_S)
sqlStr ='''create table 股票基本資料表
(
	股票基本資料表編號 nvarchar(20) primary key
	,[普通股數] nvarchar(50) NULL
	,[市場別] nvarchar(20) Null
	,[董事長] text Null
	,統計日期 nvarchar(10) not Null
	,股票代號 nvarchar(10) not Null
)
'''
cursor.execute(sqlStr)

2022/12/18


In [59]:
connsqlodbc.autocommit = True

connsqlodbc.setdecoding(pyodbc.SQL_CHAR, encoding='utf-8')
connsqlodbc.setencoding('utf-8')
cursor = connsqlodbc.cursor()
# D+代號+發周別
# MYSQL指令

Dstock_id='D'+str(stock_id)+time_S
sqlStr = f""" 
    Insert into 股票基本資料表 (股票基本資料表編號
    ,[普通股數],[市場別],[董事長],統計日期,股票代號)
    values ('{str(Dstock_id)}',
    '{products_list[11][1]}',
    '{products_list[2][3]}','{products_list[13][1]}','{time_S}','{stock_id}')
    
    """
cursor.execute(sqlStr)

print(sqlStr)

 
    Insert into 股票基本資料表 (股票基本資料表編號
    ,[普通股數],[市場別],[董事長],統計日期,股票代號)
    values ('D12182022/12/18',
    '499,999,038 股 (含私募0股)',
    '上市','詹景超','2022/12/18','1218')
    
    


In [42]:
stock_id = "6446"  #1605 1218 6446
url = 'https://goodinfo.tw/tw/StockDirectorSharehold.asp?STOCK_ID='+stock_id
payloads = {"STOCK_ID":stock_id}
headers = {"user-agent":"Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) \
Chrome/107.0.0.0 Safari/537.36"}
resp = requests.request(method="POST",url=url,data=payloads,headers=headers)
resp.encoding = "utf-8"

table_s = pd.read_html(resp.text)


In [43]:
df = table_s[15].dropna(axis='columns')
print(len(df))
df.head(2)

38


月別 當月股價              發行張數(萬張) 非獨立董監持股                          ...  \
        月別 當月收盤 漲跌(元)  漲跌(%) 發行張數(萬張)    持股張數 持股(%)  持股增減  質押張數 質押(%)  ...   
0  2022/12  525    -5   -0.9    30.25       -     -     -     -     -  ...   
1  2022/11  530   +76  +16.7    30.25   49424  16.3  -136  3617   7.3  ...   

  獨立董監持股                 全體董監持股                         外資持股(%)  
   持股(%) 持股增減 質押張數 質押(%)   持股張數 持股(%)  持股增減  質押張數 質押(%) 外資持股(%)  
0      -    -    -     -      -     -     -     -     -    12.2  
1      0    0    0     0  49522  16.4  -136  3617   7.3    11.4  

[2 rows x 21 columns]

In [44]:
products_list_2 = df.values.tolist()
print(products_list_2[19])

del products_list_2[19]
del products_list_2[18]
lists_num=len(products_list_2)
print(len(products_list_2))
print(products_list_2[1])

['月別', '當月收盤', '漲跌(元)', '漲跌(%)', '發行張數(萬張)', '持股張數', '持股(%)', '持股增減', '質押張數', '質押(%)', '持股張數', '持股(%)', '持股增減', '質押張數', '質押(%)', '持股張數', '持股(%)', '持股增減', '質押張數', '質押(%)', '外資持股(%)']
36
['2022/11', '530', '+76', '+16.7', '30.25', '49424', '16.3', '-136', '3617', '7.3', '98', '0', '0', '0', '0', '49522', '16.4', '-136', '3617', '7.3', '11.4']
